# Utilisation du modèle SLM Phi3 dans Microsoft Fabric

In [2]:
#install onnx runtime
!pip install --pre onnxruntime-genai --q
import onnxruntime_genai as og
import os
import argparse
import time
import shutil

StatementMeta(, 42990a80-5f03-4982-8221-0eaa838ec710, 4, Finished, Available, Finished)

In [8]:
model_path = '/lakehouse/default/Files/phi3slm'
if not os.path.exists(model_path) : os.mkdir(model_path)
print(model_path)

StatementMeta(, 42990a80-5f03-4982-8221-0eaa838ec710, 10, Finished, Available, Finished)

/lakehouse/default/Files/phi3slm


In [9]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-onnx --include cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/* --local-dir ./lakehouse/default/Files/phi3slm

StatementMeta(, 42990a80-5f03-4982-8221-0eaa838ec710, 11, Finished, Available, Finished)

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Fetching 10 files: 100%|████████████████████████| 10/10 [00:00<00:00, 26.71it/s]
/mnt/var/hadoop/tmp/nm-secondary-local-dir/usercache/trusted-service-user/appcache/application_1720529400292_0001/container_1720529400292_0001_01_000001/lakehouse/default/Files/phi3slm


In [10]:
#copy files to a lakehouse from the temp directory
source_dir = os.path.abspath(".") + model_path
destination_dir = model_path

try:
    shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)
    print("Directory copied successfully.")
except Exception as e:
    print(f"Error occurred: {e}")

StatementMeta(, 42990a80-5f03-4982-8221-0eaa838ec710, 12, Finished, Available, Finished)

Directory copied successfully.


In [11]:
model = og.Model('/lakehouse/default/Files/phi3slm/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()
search_options = {"max_length": 1024,"temperature":0.6}

StatementMeta(, 42990a80-5f03-4982-8221-0eaa838ec710, 13, Finished, Available, Finished)

In [12]:
params = og.GeneratorParams(model)
params.try_graph_capture_with_max_batch_size(0)
params.set_search_options(**search_options)
prompt = "<|system|>You are a famous chess player.<|end|><|user|>Could you make a comparison between chess and football ? Imagine a storyline in french :)<|end|><|assistant|>"
input_tokens = tokenizer.encode(prompt)
params.input_ids = input_tokens
generator = og.Generator(model, params)

StatementMeta(, 42990a80-5f03-4982-8221-0eaa838ec710, 14, Finished, Available, Finished)

In [13]:
while not generator.is_done():
                generator.compute_logits()
                generator.generate_next_token()

                new_token = generator.get_next_tokens()[0]
                print(tokenizer_stream.decode(new_token), end='', flush=True)

StatementMeta(, 42990a80-5f03-4982-8221-0eaa838ec710, 15, Finished, Available, Finished)

 Bien sûr, voici une histoire qui compare le chess et le football en français :

Dans un petit village pittoresque, deux amis, Pierre et Jacques, étaient connus pour leur passion pour les jeux de stratégie. Pierre était un grand maître du chess, un jeu de patience et de réflexion, où chaque coup était un écho dans le silence de la salle de jeu. Jacques, quant à lui, était un footballeur de haut niveau, un maître des mouvements rapides et des stratégies en temps réel.

Un jour, un jour, le village a décidé de créer un événement unique pour unir les deux mondes du jeu. Ils ont organisé un "Chess Football Match", une compétition où les règles du chess et du football seraient combinées pour créer une nouvelle forme de jeu.

Le match a commencé avec une équipe de chess, composée de Pierre et d'autres maîtres de l'échecs, affrontant une équipe de football, avec Jacques à sa tête. Les spectateurs, intrigués par cette combinaison inédite, ont été captivés par le spectacle.

Dans cette nouvelle